# Snowflake as Data Source for training an ML Model with Amazon Sagemaker
**_Use of Snowflake Data Table as Data Source and SageMaker JumpStart to train machine learning models without having Snowflake Data to stage on S3_**

This notebook works well with the `conda_python3` kernel on a SageMaker Notebook `ml.t3.xlarge` instance.

---
---

## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Train Amazon SageMaker XGBoost Regressor with Direct Snowflake Integration](#Train-Amazon-SageMaker-XGBoost-Regressor-with-Direct-Snowflake-Integration)
1. [Conclusion](#Conclusion)

---

## Objective

This notebook illustrates how, in just a few lines of code, [Amazon SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html) XGBoost and SKLearn models with direct data integration with [Snowflake](https://www.snowflake.com/) can be used to train an ML model on Amazon SageMaker _without having to first store the Snowflake data in S3 or write a custom training script and/or algorithm container_.

This example uses the [California Housing dataset (provided by Scikit-Learn)](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) and trains a XGBoost model to predict house prices. A detailed description about the dataset can be found [here](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html).

To understand the code, you might also find it useful to refer to:

- *The [documentation on SageMaker JumpStart's low-code Estimator class in the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.jumpstart.estimator.JumpStartEstimator)*
- *The [examples of low-code training using SageMaker JumpStart's low-code Estimator class in the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#low-code-fine-tuning-with-the-jumpstartestimator-class)*
- *The guide on [Use XGBoost with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html#)*
- *The [SageMaker reference for Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#client) (The general AWS SDK for Python, including low-level bindings for SageMaker as well as many other AWS services)*
---

## Background (Problem Description and Approach)

- **Problem statement**: SageMaker requires the training data to be present either in [S3 or in EFS or in FSX for Lustre](https://aws.amazon.com/blogs/machine-learning/choose-the-best-data-source-for-your-amazon-sagemaker-training-job/). In order to train a model using data stored outside of the three supported storage services, the data first needs to be ingested into one of these services (typically S3). This requires building a data pipeline (using tools such as [Amazon SageMaker Data Wrangler](https://aws.amazon.com/sagemaker/data-wrangler/)) to move data into S3. However, this may create a data management challenge in some situations (data lifecycle management, access control etc.) and it may be desirable to have the data accessible to SageMaker _without_ the intermediate storage of data into S3. This notebook illustrates a way to do this using Snowflake as a 3rd party data source.

- **Our approach**: Use SageMaker JumpStart to start a SageMaker Training Job in just a few lines of code. SageMaker JumpStart offers XGBoost and SciKit-Learn models `xgboost-classification-snowflake`, `xgboost-regression-snowflake`, `sklearn-classification-snowflake`, and `sklearn-regression-snowflake` that download the data from Snowflake directly into the instance created for running the training job, thus avoiding the temporary storage of data in S3. **Note that it is assumed that the data is already available in Snowflake, see [`snowflake instructions`](./snowflake-instructions.md) for instructions on creating a database in Snowflake and ingesting the California Housing dataset as a table.**

- **Our tools**: [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/) and [Amazon SageMaker JumpStart's low-code Estimator class in the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#low-code-fine-tuning-with-the-jumpstartestimator-class).


---

## Pre-requisites

Steps 1 and 2 are executed outside of this notebook. 

1. See [`snowflake instructions`](./snowflake-instructions.md) for instructions on creating a database in Snowflake and ingesting the California Housing dataset as a table.
1. See [`secrets manager instructions`](./secretsmanager-instructions.md) for instructions on storing Snowflake credentials that will be used for SageMaker Training Jobs.

In [ ]:
!pip install sagemaker==2.168.0 --upgrade

---

## Train SageMaker JumpStart XGBoost Regressor with Direct Snowflake Integration

### Environment setup

In [ ]:
%store -r sf_account_id
%store -r sf_secret_id
print(f"sf_account_id={sf_account_id}, sf_secret_id={sf_secret_id}")

In [ ]:
import boto3

# do not change!!!
# the values of these variables match what we put in the snowflake-load-dataset.ipynb file
warehouse = "amazon_sagemake_w_snowflake_as_datasource"
database = "housing"
schema = "housing_schema"
training_table = "california_housing"
session = boto3.session.Session()
region = session.region_name
print(f"region={region}")

# The environment variable SF_VALIDATION_TABLE is optional. If not specified,
# part of the training data will be used as validation.
environment = {
       "SF_ACCOUNT_ID": sf_account_id,
       "SF_SECRET_ID": sf_secret_id,
       "SF_WAREHOUSE": warehouse,
       "SF_DATABASE": database,
       "SF_SCHEMA": schema,
       "SF_TRAINING_TABLE": training_table,
       "AWS_REGION": region,
}

### Fetch SageMaker JumpStart models with direct Snowflake integration

SageMaker JumpStart now offers four new models with direct Snowflake integration:
1. `xgboost-regression-snowflake`
2. `xgboost-classification-snowflake`
3. `sklearn-regression-snowflake`
4. `sklearn-classification-snowflake`

These models can be easily trained programatically via the [JumpStartEstimator class in the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html#low-code-fine-tuning-with-the-jumpstartestimator-class).

SageMaker JumpStart and the SageMaker Python SDK make training these models easy by providing defaults for instance types and hyperparameters so users don't need to explicitly specify these parameters out of the box.*

**Users who wish to modify these defaults to their use case can do so by [modifying the arguments to the JumpStart estimator.](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.jumpstart.estimator.JumpStartEstimator)*

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.jumpstart.estimator import JumpStartEstimator

role = get_execution_role()
sm_session = sagemaker.Session()
bucket = None  # optionally specify your bucket here, eg: 'mybucket-us-east-1'; Otherwise, SageMaker will use
# the default acct bucket to upload model artifacts
if bucket is None and sm_session is not None:
    bucket = sm_session.default_bucket()
print(f"bucket={bucket}, role={role}")
prefix = "sagemaker/sagemaker-jumpstart-snowflake-example"
output_path = f"s3://{bucket}/{prefix}/housing-dist-xgb/output"

model_id = "xgboost-regression-snowflake"
# SageMaker JumpStart also offers Snowflake direct integration with these models:
# model_id = "xgboost-classification-snowflake"
# model_id = "sklearn-regression-snowflake"
# model_id = "sklearn-classification-snowflake"

# collect default subnet IDs to deploy Sagemaker training job into
ec2_session = boto3.Session(region_name=region)
ec2_resource = ec2_session.resource("ec2")
subnet_ids = []
for vpc in ec2_resource.vpcs.all():
    # here you can choose which subnet based on the id
    if vpc.is_default == True:
        for subnet in vpc.subnets.all():
            if subnet.default_for_az == True:
                subnet_ids.append(subnet.id)

# SageMaker JumpStart makes model training easy by defining defaults for instance
# types and hyperparameters so you don't have to. These parameters can still be
# modified by the user, but this is not necessary out of the box.
xgb_snowflake_estimator = JumpStartEstimator(
    model_id=model_id,
    role=role,
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=sm_session,
    environment=environment,
    subnets=subnet_ids,
)

### Train JumpStart XGBoost model with direct Snowflake integration

In [ ]:
# Estimator fitting. Data from Snowflake will be downloaded onto the compute instance during the training job
xgb_snowflake_estimator.fit()
# Optional: Deploy the model to a SageMaker Inference Endpoint with one line of code with SageMaker JumpStart
# xgb_snowflake_estimator.deploy()
# If you deploy the endpoint remember to clean it up to avoid incurring any future charges!

In [ ]:
print(f"the trained model is available in S3 -> {xgb_snowflake_estimator.model_data}")

---

## Cleaning up

To avoid incurring future charges, delete the resources. You can do this by deleting the cloud formation template used to create the IAM role and the Amazon SageMaker Notebook.
![Cleaning Up](img/cfn-delete.png)

You will have to delete the Snowflake resources manually from the Snowflake console.

---

## Conclusion

In this notebook we saw how Amazon SageMaker JumpStart can be used to quickly train an XGBoost model directly from data stored in Snowflake in just a few lines of code. **SageMaker JumpStart directly integrates Snowflake as a data source with Sagemaker Training without having the data staged on S3.**